In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv")
test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")

misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
sample_submission = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/sample_submission.csv")

In [4]:
train

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,1864,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,C,What is the range of the following numbers?\n\...,\( 5 \),\( 11 \),\( 23 \),\( 16 \),2456.0,691.0,NaN,1349.0
1865,1865,2695,"Describe an enlargement, with no centre of enl...",90,Length Scale Factors in Similar Shapes,B,Shape \( Q \) is an enlargement of shape \( P ...,\( 3 \div 11 \),\( 11 \div 3 \),\( 3 \times 11 \),\( 11-3 \),1500.0,NaN,2442.0,1258.0
1866,1866,854,Use the order of operations to carry out calcu...,33,BIDMAS,B,What does the following equal?\n\[\n8-7+10 \ti...,\( 36 \),\( 31 \),\( -29 \),\( 33 \),NaN,NaN,2306.0,1507.0
1867,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,Tom and Katie are discussing congruence and si...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,2312.0,NaN,2312.0,2312.0


In [23]:
# Nan 出ないものだけ抽出
train_A = train[train['MisconceptionAId'].notna()]
train_B = train[train['MisconceptionBId'].notna()]
train_C = train[train['MisconceptionCId'].notna()]
train_D = train[train['MisconceptionDId'].notna()]

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
5,5,2052,Identify a unit of area,75,Area of Simple Shapes,D,James has answered a question on the area of a...,\( m \),\( \mathrm{cm} \),\( \mathrm{km}^{3} \),\( \mathrm{mm}^{2} \),686.0,686.0,686.0,NaN
6,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,B,Convert this percentage to a fraction\n\( 62 \...,\( \frac{62}{10} \),\( \frac{31}{50} \),\( \frac{6}{2} \),None of these,329.0,NaN,847.0,329.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,1861,206,Round integers to the nearest 10,213,"Rounding to the Nearest Whole (10, 100, etc)",C,What is \( 326 \) rounded to the nearest \( 10...,\( 300 \),\( 320 \),\( 330 \),\( 350 \),2392.0,1379.0,NaN,2392.0
1864,1864,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,C,What is the range of the following numbers?\n\...,\( 5 \),\( 11 \),\( 23 \),\( 16 \),2456.0,691.0,NaN,1349.0
1865,1865,2695,"Describe an enlargement, with no centre of enl...",90,Length Scale Factors in Similar Shapes,B,Shape \( Q \) is an enlargement of shape \( P ...,\( 3 \div 11 \),\( 11 \div 3 \),\( 3 \times 11 \),\( 11-3 \),1500.0,NaN,2442.0,1258.0
1867,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,Tom and Katie are discussing congruence and si...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,2312.0,NaN,2312.0,2312.0
